In [5]:
import os
import re
import glob
import csv

"""
This code was written for Python 2.7, which would need updating
The structure of .xml files downloadable from NHS Choices has also changed,
which means this code is legacy code.
"""

#take out relevant data from .xml file to .csv file format
# this code was written to scrape data for all NHS institutions which
# have user feedback, including hospitals, care homes as well as GP practices
"""
Likert-scale questions' ID numbers:
0 - 'How likely are you to recommend this pharmacy to friends and family if they needed similar care or treatment?'
1 - 'How likely are you to recommend this optician to friends and family if they needed similar care or treatment?'
2 - 'How satisfied are you with the cleanliness of the area you were treated in?'
3 - 'How satisfied are you that the hospital staff worked well together?'
4 - 'How satisfied are you that you were treated with dignity and respect by the staff?'
5 - 'How satisfied are you that you were involved in decisions about your care?'
6 - 'How satisfied are you that the hospital provides same-sex accommodation?'
7 - 'How likely are you to recommend this hospital to friends and family if they needed similar care or treatment?'
8 - 'Are you able to get through to the surgery by telephone?'
9 - 'Are you able to get an appointment when you want one?'
10 - 'Do the staff treat you with dignity and respect?'
11 - 'Does the surgery involve you in decisions about your care and treatment?'
12 - 'How likely are you to recommend this GP surgery to friends and family if they needed similar care or treatment?'
13 - 'This GP practice provides accurate and up to date information on services and opening hours'
14 - 'How satisfied are you with the time you have to wait for an appointment?'
15 - 'How satisfied are you that you are you treated with dignity and respect by staff?'
16 - 'How satisfied are you that the dental surgery involves you in decisions about your treatment?'
17 - 'How satisfied are you with the information given by the surgery on the cost of your NHS treatment?'
18 - 'How satisfied are you with the outcome of your treatment?'
19 - 'How likely are you to recommend this dentist to friends and family if they needed similar care or treatment?'
20 - 'The environment where I was treated was?'
21 - 'The Clinic  staff worked well together?'
22 - 'I was treated with dignity and respect by the Clinic  staff?'
23 - 'I was involved with decisions about my care?'
24 - 'Friends and Family'
25 - 'How likely are you to recommend this care provider to friends and family if they needed similar care or treatment?'
26 - 'NHS Clinic s must provide same-sex accommodation. How satisfied were you that this Clinic  did so?'
27 - 'Overall recommendation '
"""

foldery = ["gp practices 2016"]
wiersz_zero = ["MyDatabaseID", "category", "NHS_review_ID", "type", "commenter"]
pytania =['How likely are you to recommend this pharmacy to friends and family if they needed similar care or treatment?', 'How likely are you to recommend this optician to friends and family if they needed similar care or treatment?', 'How satisfied are you with the cleanliness of the area you were treated in?', 'How satisfied are you that the hospital staff worked well together?', 'How satisfied are you that you were treated with dignity and respect by the staff?', 'How satisfied are you that you were involved in decisions about your care?', 'How satisfied are you that the hospital provides same-sex accommodation?', 'How likely are you to recommend this hospital to friends and family if they needed similar care or treatment?', 'Are you able to get through to the surgery by telephone?', 'Are you able to get an appointment when you want one?', 'Do the staff treat you with dignity and respect?', 'Does the surgery involve you in decisions about your care and treatment?', 'How likely are you to recommend this GP surgery to friends and family if they needed similar care or treatment?', 'This GP practice provides accurate and up to date information on services and opening hours', 'How satisfied are you with the time you have to wait for an appointment?', 'How satisfied are you that you are you treated with dignity and respect by staff?', 'How satisfied are you that the dental surgery involves you in decisions about your treatment?', 'How satisfied are you with the information given by the surgery on the cost of your NHS treatment?', 'How satisfied are you with the outcome of your treatment?', 'How likely are you to recommend this dentist to friends and family if they needed similar care or treatment?', 'The environment where I was treated was?', 'The Clinic  staff worked well together?', 'I was treated with dignity and respect by the Clinic  staff?', 'I was involved with decisions about my care?', 'Friends and Family', 'How likely are you to recommend this care provider to friends and family if they needed similar care or treatment?', 'NHS Clinic s must provide same-sex accommodation. How satisfied were you that this Clinic  did so?', 'Overall recommendation ']
tymcz = (len(pytania) + 1)*3 
list_pyt = range(tymcz)
list_pyt = [ "" for a in list_pyt]
rodzaje_uslug = []

start = 1
for i in foldery:
    i = 'c:\\users\\user\\desktop' + '\\' + i
    os.chdir(i)
    for filer in glob.glob("*.txt"):
        with open(filer, "r") as f:
            okej = f.read()
            start_lista = [a.start() for a in re.finditer('<item>', okej)]
            koniec_lista = [a.end() for a in re.finditer('</item>', okej)]
            komentarze = []
        #     identify where in .xml are the written comments
            for i, value in enumerate(start_lista):
                komentarz = []
                komentarz.append(start_lista[i])
                komentarz.append(koniec_lista[i])
                komentarze.append(komentarz)

        #   CONSTRUCTING A LIST OF ITEMS WHICH WILL COMPRISE A ROW OF DATA IN A .csv FILE
            # 1 ROW = 1 COMMENT
            for indeks, punkty_start_koniec in enumerate(komentarze):
                tresc = okej[punkty_start_koniec[0]:punkty_start_koniec[1]]

        #     first element. Review ID
                wiersz = []
                ind2016 = str(start) + str(".2016")
                wiersz.append(ind2016)
                start = start + 1

        #     second element. NHS institution type
        #"http://v1.syndication.nhschoices.nhs.uk/organisations/gppractices/commentssince/2016/1/1?apikey=JROLO&amp;page=2"
                alfa = "http://v1.syndication.nhschoices.nhs.uk/organisations/"
                beta = "/commentssince/2016/1/1?apikey=JROLO"
                if alfa and beta in okej:
                    jeden = okej.index(alfa)
                    dwa = okej.index(beta)
                    digit = jeden + len(alfa)
                    wynik = okej[digit:dwa]
                    wiersz.append(wynik)

        # third element: NHS ID number for the review
                a = tresc.find('<guid isPermaLink="false">') + len('<guid isPermaLink="false">')
                b = tresc.find("</guid>")
                fin = str(tresc[a:b])
                znak = fin.find("R" or "r")
                znak = str(znak)
                if len(znak) == 2:
                    wiersz.append(fin)
                else:
                    fin = fin[:-1]
                    wiersz.append(fin)

        # fourth element: comment type (deletion, reply lub comment)
                a = tresc.find('<category domain="commentType">') + len('<category domain="commentType">')
                b = tresc.find("</category>")
                wiersz.append(tresc[a:b])

        # fifth element: commented-on institution's ID number in the database (it's not the same as ODS code)
                if tresc[a:b] != "deletion":
                    a = tresc.find('aspx?id=') + len('aspx?id=')
                    b = tresc.find("</link>")
                    wiersz.append(tresc[a:b])

        # sixth element: comment author
                    a = tresc.find('<a10:author><a10:name>') + len('<a10:author><a10:name>')
                    b = tresc.find("</a10:name></a10:author>")
                    fin = tresc[a:b]
                    if fin != "Anonymous":
                        wiersz.append(tresc[a:b])
                    else:
                        wiersz.append("nan")
                        
        # comment title
                    a = tresc.find('<title>') + len('<title>')
                    b = tresc.find("</title>")
                    fin = str(tresc[a:b])
                    if fin != 'Rating without comment':
                        wiersz.append(fin)
                    else:
                    # dodawanie również ósmego elementu, jeśli nie było komentarza
                        wiersz.extend(['nan', 'nan'])
                    
        # comment text
                    a = tresc.find('[CDATA[') + len('[CDATA[')
                    b = tresc.find("]]&gt")
                    if len(wiersz) == 7:
                        wiersz.append(tresc[a:b])
                    else:
                        pass


        # time stamp
                    a = tresc.find('<pubDate>') + len('<pubDate>')
                    b = tresc.find("</pubDate>")
                    dni_tygodnia = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
                    miesiace = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
                    a = tresc.find('<pubDate>') + len('<pubDate>')
                    b = tresc.find("</pubDate>")
                    fin = tresc[a:b]
                    # dzień tygodnia
                    dzien_tygodnia = fin[:3]
                    for i, w in enumerate(dni_tygodnia):
                        if dzien_tygodnia != w:
                            pass
                        else:
                            dzien_tygodnia = "0" + str(i + 1)
                            wiersz.append(dzien_tygodnia)
                    # dzień miesiąca     
                    dzien_miesiaca = fin[5:7]
                    wiersz.append(dzien_miesiaca)
                    # miesiąc
                    miesiac = fin[8:11]
                    for i, w in enumerate(miesiace):
                        if miesiac != w:
                            pass
                        else:
                            miesiac = "0" + str(i+1)
                            if len(miesiac) == 2:
                                wiersz.append(miesiac)
                            else:
                                miesiac = miesiac[1:]
                                wiersz.append(miesiac)
                    # rok
                    rok = fin[12:16]
                    wiersz.append(rok)

        # html link to data page of commented-on NHS institution
                    if wiersz[1] != 'careproviders':
                        a = tresc.find('<a10:link rel="related" type="application/xhtml+xml" title="Organisation commented on" length="1000" href="') + len('<a10:link rel="related" type="application/xhtml+xml" title="Organisation commented on" length="1000" href="')
                        b = tresc[a:].find("?apikey")
                        wiersz.append(tresc[a:(a+b)])
                    else:
                        wiersz.append('nan')

        # posting organisation ID (meaning of this piece of information is unknown)
                    a = tresc.find('<postingorganisationid xmlns="http://syndication.nhschoices.nhs.uk/schemas/comments">') + len('<postingorganisationid xmlns="http://syndication.nhschoices.nhs.uk/schemas/comments">')
                    b = tresc.find("</postingorganisationid>")
                    wiersz.append(tresc[a:b])

        # Likert-scale Q&A
        
                    pocz = [a.end() for a in re.finditer('<rating>', tresc)]
                    kon = [a.start() for a in re.finditer('</rating>', tresc)]
                    list_pyt2 = list(list_pyt)
                    for ind, wart in enumerate(pocz):
                        pytanie = tresc[(wart):kon[ind]]
                        a = pytanie.find('<questionText>') + len('<questionText>')
                        b = pytanie.find("</questionText>")
                        fin = pytanie[a:b]
                        for i, row in enumerate(pytania):
                            if fin != row:
                                pass
                            else:
                                indeks = (i + 1)*3
                                list_pyt2[indeks] = indeks/3 
                                a = pytanie.find('<answerText>') + len('<answerText>')
                                b = pytanie.find("</answerText>")
                                fin = pytanie[a:b]
                                list_pyt2[(indeks+1)] = fin
                                a = pytanie.find('<answerMetric value="') + len('<answerMetric value="')
                                odpowiedz = pytanie[a:(a+4)]
                                b = odpowiedz.find('"')
                                cyfra = odpowiedz[:b]
                                list_pyt2[(indeks+2)] = cyfra
                    wiersz.extend(list_pyt2[3:])
#                     del list_pyt2



                    listek = [a.end() for a in re.finditer('<a10:link rel="related"', tresc)]
                    miniwiersz = []
                    for r, val in enumerate(listek):
                        val = val + 37
                        minitekst = tresc[val:(val+100)]
                        koniec = minitekst.find('"')
                        finisz = minitekst[:koniec]
                        if r != 0:
                            miniwiersz.append(finisz)
                        else:
                            pass
                    

                    kody = []
                    for usluga in miniwiersz:
                        if usluga not in rodzaje_uslug:
                            rodzaje_uslug.append(usluga)
                            kody.append(rodzaje_uslug.index(usluga))
                        else:
                            kody.append(rodzaje_uslug.index(usluga))
                # putting together final list to be entered into .csv file
                    fin = []
                    for indeks, wartosc in enumerate(rodzaje_uslug):
                        if indeks not in kody:
                            fin.append("")
                        else:
                            fin.append(1)
                    wiersz.extend(fin)
                    del fin
                    
                    
                else:
                    # information on deleted posts from NHS database
                    wiersz.extend(['nan', 'nan', 'nan', 'nan'])
                    a = tresc.find('<pubDate>') + len('<pubDate>')
                    b = tresc.find("</pubDate>")
                
                    dni_tygodnia = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
                    miesiace = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
                    a = tresc.find('<pubDate>') + len('<pubDate>')
                    b = tresc.find("</pubDate>")
                    fin = tresc[a:b]

                    # day
                    dzien_tygodnia = fin[:3]
                    for i, w in enumerate(dni_tygodnia):
                        if dzien_tygodnia != w:
                            pass
                        else:
                            dzien_tygodnia = "0" + str(i + 1)
                            wiersz.append(dzien_tygodnia)
                    # day of month 
                    dzien_miesiaca = fin[5:7]
                    wiersz.append(dzien_miesiaca)
                    # month
                    miesiac = fin[8:11]
                    for i, w in enumerate(miesiace):
                        if miesiac != w:
                            pass
                        else:
                            miesiac = "0" + str(i+1)
                            if len(miesiac) == 2:
                                wiersz.append(miesiac)
                            else:
                                miesiac = miesiac[1:]
                                wiersz.append(miesiac)
                    # year
                    rok = fin[12:16]
                    wiersz.append(rok)

                with open("c:\\users\\user\\desktop\\gp practices 2016\\final2016.csv", "ab") as final:
                    a = csv.writer(final, delimiter=',')
                    a.writerow(wiersz)

os.chdir('c:\\users\\user\\desktop')

# list of additional services which NHS institutions may offer (printed below are possibilities for GP practices)
for numer, usluga in enumerate(rodzaje_uslug):
    print str(numer + 1) + " " + str(usluga)

print "koniec"

1 Asthma Clinic
2 Child Immunisations
3 COPD clinic with spirometry
4 Diabetes Clinic providing insulin initiation
5 Minor surgery (e.g. removal of moles and skin lesions) - provided in-house
6 Learning disability health check
7 Baby clinic with health visitor
8 Dermatology specialist clinic
9 Phlebotomy
10 Smoking cessation clinic
11 Travel health with yellow fever
12 Primary care counselling service
13 Obesity management clinic
14 Urgent care centre
15 Walk-in centre
16 Long-Acting Reversible Contraception (LARC - eg IUD or implant)
17 Vasectomy
18 Anticoagulant monitoring and dosing - provided in-house
19 Child health and development
20 Travel health without yellow fever
21 Physiotherapy - provided in house and available on NHS
22 Acupuncture
23 Joint injections
24 Pharmacy available on-site
25 Dressings clinic (nurse led)
26 Ear, Nose and Throat (ENT) specialist clinic
27 Young person's clinic
28 Chiropody - provided in house and available on NHS
29 Dietician - provided in house an